In [38]:
%%time
import pandas as pd

from sklearn.metrics import roc_auc_score, make_scorer, accuracy_score
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler


from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from xgboost import XGBClassifier
from sklearn.feature_selection import RFE


from time import strftime, strptime, mktime, time, ctime

from sklearn import model_selection

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB 
from sklearn.svm import LinearSVC

from xgboost import XGBClassifier
import xgboost as xgb

from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier,\
ExtraTreesClassifier, GradientBoostingClassifier, VotingClassifier, RandomForestClassifier

from mlxtend.classifier import StackingClassifier, StackingCVClassifier
from mlxtend.feature_selection import ColumnSelector

from sklearn.pipeline import make_pipeline
import numpy as np

import gc


CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 59.1 µs


In [65]:
%%time
df = pd.read_csv('./train.csv')
print df.shape
test = pd.read_csv('./test.csv')
print test.shape

y = df.pop('Approved')
train_id = df.pop('ID')
test_id = test.pop('ID')

def helper(x):
    try:
        tmp = strptime(x, '%d/%m/%y')
        tmp = mktime(tmp)
    except:
        tmp = 563000400
    return tmp

df['Lead_Creation_Date'] = df['Lead_Creation_Date'].apply(helper)
test['Lead_Creation_Date'] = test['Lead_Creation_Date'].apply(helper)

df['DOB'] = df['DOB'].apply(helper)
test['DOB'] = test['DOB'].apply(helper)

df = df.fillna(axis=1, value=-1)
test = test.fillna(axis=1, value=-1)

df = pd.get_dummies(df)
test = pd.get_dummies(test)
test = test.reindex(columns = df.columns, fill_value=0)

test_o = test.copy()

(69713, 22)
(30037, 21)
CPU times: user 10.2 s, sys: 3.14 s, total: 13.4 s
Wall time: 13.4 s


In [61]:
%%time

rid = []

for i in df:
    if df[i].value_counts().head(1).values[0]>df.shape[0]*0.995:
        rid.append(i)
print len(rid), df.shape[1]-len(rid)

37323 32390
CPU times: user 35.8 s, sys: 41 ms, total: 35.9 s
Wall time: 35.9 s


In [62]:
df.shape[1]-len(rid)

93

In [70]:
%%time
test = test_o.copy()
train = df.drop(rid, axis=1)
test = test.drop(rid, axis=1)

CPU times: user 957 ms, sys: 1.32 s, total: 2.27 s
Wall time: 2.27 s


In [71]:
%%time
s = StandardScaler()
train = s.fit_transform(train)
test = s.transform(test)

print train.shape, test.shape

(69713, 93) (30037, 93)
CPU times: user 109 ms, sys: 72 ms, total: 181 ms
Wall time: 180 ms


### from here

In [72]:
%%time
m = lambda *x: [i.shape for i in x]
X_train, X_test, y_train, y_test = train_test_split(train, y, test_size=0.2)
print m(X_train, y_train, X_test, y_test)

[(55770, 93), (55770,), (13943, 93), (13943,)]
CPU times: user 47 ms, sys: 26 ms, total: 73 ms
Wall time: 71.3 ms


In [73]:
%%time
d_train = xgb.DMatrix(X_train, label=y_train)
d_test = xgb.DMatrix(X_test, label=y_test)

CPU times: user 69 ms, sys: 16 ms, total: 85 ms
Wall time: 84.7 ms


In [76]:
RANDOM_STATE = 2018
params = {
    'min_child_weight': 0.8,
#     'eta': 0.1,
    'colsample_bytree': 0.8,
    'max_depth': 12,
    'subsample': 0.8,
    'alpha': 0.3,
    'lambda':0.9,
    'gamma': 0.3,
    'silent': 0,
    'learning_rate':0.02,
    'verbose_eval': True,
    'seed': RANDOM_STATE,
    'objective':"binary:logistic",
#     'eval_metric':'error'
#     'num_class':1
    'n_jobs':24,
    'eval_metric':'auc'
    
}
watchlist = [(d_train, 'train'), (d_test, 'test')]



In [77]:
start=time()

clf = xgb.train(params, d_train, 10000, watchlist, early_stopping_rounds=100, 
                verbose_eval=50,  maximize=True,
               )
                


'!!! used %.2f s'%(time()-start)

[0]	train-auc:0.78608	test-auc:0.774681
Multiple eval metrics have been passed: 'test-auc' will be used for early stopping.

Will train until test-auc hasn't improved in 100 rounds.
[50]	train-auc:0.869057	test-auc:0.820646
[100]	train-auc:0.917223	test-auc:0.832724
[150]	train-auc:0.954695	test-auc:0.841354
[200]	train-auc:0.97324	test-auc:0.847677
[250]	train-auc:0.983956	test-auc:0.850927
[300]	train-auc:0.990142	test-auc:0.854498
[350]	train-auc:0.993693	test-auc:0.85648
[400]	train-auc:0.995984	test-auc:0.856517
[450]	train-auc:0.997369	test-auc:0.855806
[500]	train-auc:0.99827	test-auc:0.855364
Stopping. Best iteration:
[411]	train-auc:0.996312	test-auc:0.857112



'!!! used 14.39 s'

In [78]:
%%time
dtest = xgb.DMatrix(test)

CPU times: user 36 ms, sys: 9 ms, total: 45 ms
Wall time: 43.4 ms


In [79]:
%%time
predb = clf.predict(dtest)


CPU times: user 2.2 s, sys: 1 ms, total: 2.2 s
Wall time: 126 ms


___

In [80]:
submit = pd.concat([pd.DataFrame(test_id.values, columns=['ID']), pd.DataFrame(predb, columns=['Approved'])], axis=1)
submit.shape

(30037, 2)

In [81]:
submit.to_csv('xgb_rm2.csv',index=False)

#### lb .852388

In [60]:
# clf.save_model('xgb_0.852586.model')

In [441]:
roc_auc_score?

In [442]:
submit2 = pd.concat([pd.DataFrame(test_id.values, columns=['ID']), pd.DataFrame(pred, columns=['Approved'])], axis=1)
submit2.shape

(30037, 2)

In [449]:
accuracy_score(y_test, [0]*y_test.shape[0])

0.9834325467976762

___

In [276]:
cross_val_score(RandomForestClassifier(n_estimators=50, n_jobs=24), df.iloc[:,:-1], df.iloc[:,-1], cv=5)

array([0.87807502, 1.        , 0.99992828, 1.        , 0.98759145])

In [76]:
fr = RandomForestClassifier(n_estimators=50)
fr

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [77]:
fr.fit(df.iloc[:,:-1], df.iloc[:,-1])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [ ]:
fr.predict_log_proba